In [1]:
%load_ext autoreload
%autoreload 2

import os
import time

import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

sys.path.append('/oasis/projects/nsf/csd181/yuncong/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from joblib import Parallel, delayed

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from skimage.filters.rank import entropy
from skimage.morphology import remove_small_objects, disk
from skimage.measure import label, regionprops
from skimage.color import rgb2gray
from skimage.io import imread, imsave
from skimage import img_as_float

from sklearn import mixture

from skimage.measure import find_contours
from annotation_utilities import *
from skimage.segmentation import active_contour
from skimage.filters import gaussian
from skimage.filter import threshold_adaptive, canny

Setting environment for Gordon


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [3]:
# def generate_mask(img):

#     h, w = img.shape
    
#     e = entropy(img, disk(5))
    
#     clf = mixture.GMM(n_components=2, covariance_type='full')
#     clf.fit(np.atleast_2d(e[e > 0.1]).T)

#     means = np.squeeze(clf.means_)

#     order = np.argsort(means)
#     means = means[order]

#     covars =np.squeeze(clf.covars_)
#     covars = covars[order]

#     weights = clf.weights_
#     weights = weights[order]

#     counts, bins = np.histogram(e.flat, bins=100, density=True);

#     gs = np.array([w * 1./np.sqrt(2*np.pi*c) * np.exp(-(bins-m)**2/(2*c)) for m, c, w in zip(means, covars, weights)])

#     thresh = bins[np.where(gs[-1] - gs[-2] < 0)[0][-1]]

#     v = e > thresh

#     l = label(v, background=0)
#     mask = l == np.argmax([p.area for p in regionprops(l+1)])
    
#     mask = ~remove_small_objects(~mask, min_size=10000, connectivity=8)
    
#     l = label(v)
#     l[v > 0] = -1
#     props = regionprops(l)
    
#     border_holes = np.where([np.any(p.coords[:,0] == 0) or np.any(p.coords[:,1] == 0) \
#                              or np.any(p.coords[:,0] == h-1) or np.any(p.coords[:,1] == w-1) 
#                              for p in props])[0]

#     for i in border_holes:
#         c = props[i].coords
#         mask[c[:,0], c[:,1]] = 0
    
#     return mask

In [5]:
def generate_mask(fn):

    img_rgb = imread(os.path.join(input_dir, fn + '.tif'))
    img = rgb2gray(img_rgb)

    entropy_mask = generate_entropy_mask(img)

    init_contours = [yxs[:,::-1] for yxs in find_contours(entropy_mask, .5)]
    assert len(init_contours) > 0, 'No contour is detected from entropy mask %s' % fn

    img_adap = threshold_adaptive(img, 51)
    img_adap[~entropy_mask] = 1

    init_cnt = init_contours[0]
    snake = active_contour(img_adap, init_cnt,
                           alpha=1., beta=10., gamma=0.01,
                           w_line=0., w_edge=1.,
                           max_iterations=100)

    bg = np.zeros(img.shape[:2], bool)
    xys = points_inside_contour(snake.astype(np.int))
    bg[xys[:,1], xys[:,0]] = 1

    final_mask = bg & entropy_mask

    return final_mask

In [6]:
stack = 'MD585'

In [ ]:
from glob import glob

In [ ]:
stack_masks_dir = create_if_not_exists('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_mask_unsorted' % dict(stack=stack))

all_images = glob('/home/yuncong/CSHL_data/%(stack)s/*.tif' % dict(stack=stack))

bar = show_progress_bar(1, len(all_images))

for i, fn in enumerate(all_images):
    
    bar.value = i
    
    img_rgb = imread(fn)    
    img = rgb2gray(img_rgb)

    entropy_mask = generate_mask(img)

    init_contours = [yxs[:,::-1] for yxs in find_contours(entropy_mask, .5)]

    img_adap = threshold_adaptive(img, 51)
    img_adap[~entropy_mask] = 1

    init_cnt = init_contours[0]

#     t = time.time()

    # alpha: length
    # beta: second order

    snake = active_contour(img_adap, init_cnt, 
                           alpha=1., beta=10., gamma=0.01, 
                           w_line=0., w_edge=1., 
                           max_iterations=100)
    #                        convergence=.01)

#     print time.time() - t

    bg = np.zeros(img.shape[:2], bool)
    xys = points_inside_contour(snake.astype(np.int))
    bg[xys[:,1], xys[:,0]] = 1

    
    final_mask = bg & entropy_mask
    
#     final_contours = find_contours(final_mask, .5)
#     if len(final_contours) == 0:
#         sys.stderr.write('%d contour error.\n' % s)
#         continue
    
#     final_cnt = final_contours[0][:,::-1]
    
    mask_fn = os.path.join(stack_masks_dir, '%(fn)s_mask.png' % dict(fn=os.path.splitext(os.path.basename(fn))[0]))
    imsave(mask_fn, img_as_ubyte(final_mask))
        
#     fig = plt.figure(figsize=(30, 30));
#     plt.imshow(img_adap, cmap=plt.cm.gray)
#     plt.plot(init_cnt[:, 0], init_cnt[:, 1], '--r', lw=3)
#     plt.plot(snake[:, 0], snake[:, 1], '-g', lw=3)
#     plt.plot(final_cnt[:, 0], final_cnt[:, 1], '-b', lw=3)
#     plt.title(s)
#     plt.show()

#     img_rgb_lossless = imread('/home/yuncong/CSHL_data_processed/MD603/MD603_lossless_sorted_aligned_cropped_compressed/MD603_%04d_lossless_aligned_cropped_compressed.jpg' % s)
#     contour_lossless = final_cnt * 32
#     contour_lossless_cropped = np.c_[contour_lossless[:, 0]-xmin*32, contour_lossless[:, 1]-ymin*32].astype(np.int)
#     img2 = img_rgb_lossless.copy()
#     for x, y in contour_lossless_cropped:
#         cv2.circle(img2, (x,y), 10, (255,0,0), -1)

In [ ]:
with open('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_sorted_filenames.txt'%dict(stack=stack), 'r') as f:
    fn_idx_tuples = [line.strip().split() for line in f.readlines()]
    filename_to_section = {fn: int(idx) for fn, idx in fn_idx_tuples}
    
import cPickle as pickle
Ts = pickle.load(open('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_elastix_output/%(stack)s_transformsTo_anchor.pkl' % dict(stack=stack), 'r'))

Ts_inv = {}
for fn, T0 in Ts.iteritems():
    T = T0.copy()
    T[:2, 2] = T[:2, 2]
    Tinv = np.linalg.inv(T)
    Ts_inv[fn] = Tinv

In [ ]:
with open(os.path.join(data_dir, stack, stack+'_anchor.txt'), 'r') as f:
    anchor_fn = str(f.readline().strip())

In [ ]:
stack_masks_aligned_dir = create_if_not_exists('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_mask_unsorted_alignedTo_%(anchor_fn)s' %\
                        dict(stack=stack, anchor_fn=anchor_fn))

In [ ]:
for fn, sec in filename_to_section.iteritems():
    
    T = Ts_inv[fn]
    
    d = {'sx':T[0,0],
     'sy':T[1,1],
     'rx':T[1,0],
     'ry':T[0,1],
     'tx':T[0,2],
     'ty':T[1,2],
     'input_fn': os.path.join(stack_masks_dir, fn + '_mask.png'),
     'output_fn': os.path.join(stack_masks_aligned_dir, fn + 'mask_aligned.png'),
     'x': '+' + str(x * scale_factor) if int(x)>=0 else str(x * scale_factor),
     'y': '+' + str(y * scale_factor) if int(y)>=0 else str(y * scale_factor),
     'w': str(w * scale_factor),
     'h': str(h * scale_factor),
    }

    os.system("convert %(input_fn)s -virtual-pixel background +distort AffineProjection '%(sx)f,%(rx)f,%(ry)f,%(sy)f,%(tx)f,%(ty)f' -crop %(w)sx%(h)s%(x)s%(y)s\! -flatten -compress lzw %(output_fn)s"%d)